# Assignment Lesson 10:  Frequent Itemset Mining
In this assignment, we will explore the python package [mlxtend](https://pypi.org/project/mlxtend/) to mine frequent itemsets. You will learn to create association rules.
<br>
Original version found in MLEARN 510 Canvas. Updated and modified by Ernst Henle
<br>
Copyright © 2024 by Ernst Henle 

# Learning Objectives:
Learning Objectives:
- Extract frequent patterns given a dataset of transactions
- Find the association rules which are interesting and non-obvious

In [ ]:
#!pip install mlxtend==0.23.1 # install older version; version 0.23.3 has a bug
# Expected result should inclde:  Successfully installed mlxtend-0.23.1

In [ ]:
# There seems to be a bug in mlxtend version 0.23.3 and maybe mlxtend 0.23.2. 
# We need to use mlxtend version 0.23.1

import importlib.metadata
from packaging.version import parse

def check_version():
    try:
        installed_version = importlib.metadata.version("mlxtend")
    except importlib.metadata.PackageNotFoundError:
        print('\n\nPackage not found! Have you installed mlxtend?')
        print('You may need to run:\n!pip install mlxtend==0.23.1\n\n')
        return;
    max_version = "0.23.1"
    if parse(installed_version) > parse(max_version):
        print(f"Installed mlxtend version {installed_version} may have a bug.")
        print(f"Uninstall mlxtend version {installed_version}:  !pip uninstall mlxtend -y")
        print(f"Install mlxtend version {max_version}:  !pip install mlxtend==0.23.1")
    else:
        print(f"Installed mlxtend version {installed_version} is OK.  Some versions, like 0.23.3 have a bug. Version {max_version} is recommended.")

check_version()

# !pip uninstall mlxtend -y # remove current version
# !pip install mlxtend==0.23.1 # install older version
# Expected result should inclde:  Successfully installed mlxtend-0.23.1

In [ ]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

import matplotlib.pyplot as plt

## Online Retail Data
Transactional [online retail](https://archive.ics.uci.edu/ml/datasets/Online+Retail+II) dataset :  multiple items are purchased together under the same Invoice No


In [ ]:
# Read Pandas Pickled Data
file_path = '../data/online_retail_combined.pkl'  # 52763 kb file with 1067371 rows
data = pd.read_pickle(file_path) # read time is 1 sec

## Question 1.1
EDA
- show first few rows
- verify data types
- describe numeric values

<br><br>
Basic Data Preparation
- Remove rows where Quantity is Negative or above 99th percentile of Quantity
- Remove rows where Price is Negative or above 99th percentile of price

In [ ]:
display(data.head())
display(data.dtypes) # columns are properly typed
data.describe()

In [ ]:
Quantity_Hi = np.quantile(data['Quantity'], 0.99)
Quantity_Lo = 0
Price_Hi = np.quantile(data['Price'], 0.99)
Price_Lo = 0

# Remove any cases where Quantity < 0.  Such data is not easily used in our analysis
# data starts with 1067371 rows
data = data[data['Quantity'] > Quantity_Lo] # results in 1044421 rows
data = data[data['Quantity'] < Quantity_Hi] # results in 1030595 rows
data = data[data['Price'] > Price_Lo] # results in 1028147 rows
data = data[data['Price'] < Price_Hi] # results in 1017274 rows

## Question 1.2
Consistency
- Verify that each invoice is only from one country (Group Countries by 'invoice' and aggregate with `.nunique()` the number of unique countries)
- Show that Description and StockCode have a many-to-many relationship
- Enforce a 1-to-1 relationship between Description and StockCode

In [ ]:
print(f'Total number of Invoices: {data.Invoice.unique().shape[0]}')
NumberOfCountriesByInvoice = data.groupby('Invoice')['Country'].nunique()
NumberOfInvoicesWithOneCountry = len(NumberOfCountriesByInvoice[NumberOfCountriesByInvoice == 1])
print(f'Number of invoices with only one country: {NumberOfInvoicesWithOneCountry}')

In [ ]:
# Show that Description and StockCode have a many-to-many relationship
display(data.groupby('Description')['StockCode'].nunique().value_counts())
display(data.groupby('StockCode')['Description'].nunique().value_counts())

In [ ]:
# Enforce 1-to-1 relationship of 'Description' and 'StockCode'
from collections import Counter

# Find most common value from each group of values
def most_common_description(group):
    return Counter(group).most_common(1)[0][0]

# Create dictionary of StockCode to most common Description
common_descriptions = data.groupby('StockCode')['Description'].apply(most_common_description).to_dict()
# Replace Description most common Description for each StockCode
data['Description'] = data['StockCode'].map(common_descriptions)

# Create dictionary of Description to most common StockCode
common_stockcodes = data.groupby('Description')['StockCode'].apply(most_common_description).to_dict()
# Replace Description most common Description for each StockCode
data['StockCode'] = data['Description'].map(common_stockcodes)

display(data.groupby('Description')['StockCode'].nunique().value_counts())
display(data.groupby('StockCode')['Description'].nunique().value_counts())

## Question 1.2
Filter the data by only transactions that happened in the United Kingdom 

In [ ]:
data['Country'].value_counts()

In [ ]:
UK_Data = data[data['Country']=='United Kingdom'] # 939491
UK_Data.shape

## Question 1.3
What are the most popular 5 items in the United Kingdom?

In [ ]:
item_freq = UK_Data['Description'].value_counts()
display(item_freq.head(5))
item_freq_SC = UK_Data['StockCode'].value_counts()

## Question 1.4
Filter down the data to only include transactions that contain at least one of the top 20 items <br>
This does not mean there will only be 20 items.  Just that any invoice must contain at leastone of the top 20 items.

In [ ]:
RowsWithTop20Items = UK_Data['StockCode'].astype(str).isin(item_freq_SC.head(20).index.to_list())
RowsWithTop20Items.sum()

In [ ]:
print(f'Original shape of UK data: {UK_Data.shape}')
InvoicesOfInterest = UK_Data['Invoice'].isin(UK_Data.loc[RowsWithTop20Items,'Invoice'].to_list())
UK_Data_of_Interest = UK_Data.loc[InvoicesOfInterest,:]
print(f'Shape of interesting UK data: {UK_Data_of_Interest.shape}')

## Question 2.1
Create Input Data for Apriori Function:  
- Create an array of lists where each list contains items from one invoice
- Create a 2D data structure where each product description is a column and each transaction is a row
- The cell values indicate the presence (True) or absence (False) of the value

<br><br>
The result should be 1 transaction per row and each product one-hot encoded.

In [ ]:
# Aggregate Descriptions for each invoice into its own list


In [ ]:
# Create array of transaction lists 


## Question 2.2
Create table with booleans (one-hot encoding) where each invoice is a row and each description is a column.  Use `TransactionEncoder` from `mlxtend`

In [ ]:
# One hot encode transactions 


## Question 3.1 
Apply [apriori](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/) algorithm to generate frequent item sets that have a support of at least 7%

In [ ]:
# Create frequent itemsets


## Question 3.2
Generate the association rules with their corresponding support, confidence and lift.

In [ ]:
# Number of transactions in original input data


## Question 4
Bonus Question:  Based on the above rules, identify what would be the opportunity of promoting one of the antecendents.